In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
print(os.listdir("../input/cell_images/cell_images"))

In [ ]:
infected = os.listdir('../input/cell_images/cell_images/Parasitized/') 
uninfected = os.listdir('../input/cell_images/cell_images/Uninfected/')

In [ ]:
data = []
labels = []

for i in infected:
    try:
    
        image = cv2.imread("../input/cell_images/cell_images/Parasitized/"+i)
        image_array = Image.fromarray(image , 'RGB')
        resize_img = image_array.resize((50 , 50))
        rotated45 = resize_img.rotate(45)
        rotated75 = resize_img.rotate(75)
        blur = cv2.blur(np.array(resize_img) ,(10,10))
        data.append(np.array(resize_img))
        data.append(np.array(rotated45))
        data.append(np.array(rotated75))
        data.append(np.array(blur))
        labels.append(1)
        labels.append(1)
        labels.append(1)
        labels.append(1)
        
    except AttributeError:
        print('')
    
for u in uninfected:
    try:
        
        image = cv2.imread("../input/cell_images/cell_images/Uninfected/"+u)
        image_array = Image.fromarray(image , 'RGB')
        resize_img = image_array.resize((50 , 50))
        rotated45 = resize_img.rotate(45)
        rotated75 = resize_img.rotate(75)
        data.append(np.array(resize_img))
        data.append(np.array(rotated45))
        data.append(np.array(rotated75))
        labels.append(0)
        labels.append(0)
        labels.append(0)
        
    except AttributeError:
        print('')

In [ ]:
cells = np.array(data)
labels = np.array(labels)

np.save('Cells' , cells)
np.save('Labels' , labels)

In [ ]:
print('Cells : {} | labels : {}'.format(cells.shape , labels.shape))

In [ ]:
plt.figure(1 , figsize = (15 , 9))
n = 0 
for i in range(49):
    n += 1 
    r = np.random.randint(0 , cells.shape[0] , 1)
    plt.subplot(7 , 7 , n)
    plt.subplots_adjust(hspace = 0.5 , wspace = 0.5)
    plt.imshow(cells[r[0]])
    plt.title('{} : {}'.format('Infected' if labels[r[0]] == 1 else 'Unifected' ,
                               labels[r[0]]) )
    plt.xticks([]) , plt.yticks([])
    
plt.show()

In [ ]:
plt.figure(1, figsize = (15 , 7))
plt.subplot(1 , 2 , 1)
plt.imshow(cells[0])
plt.title('Infected Cell')
plt.xticks([]) , plt.yticks([])

plt.subplot(1 , 2 , 2)
plt.imshow(cells[60000])
plt.title('Uninfected Cell')
plt.xticks([]) , plt.yticks([])

plt.show()

In [ ]:
n = np.arange(cells.shape[0])
np.random.shuffle(n)
cells = cells[n]
labels = labels[n]

In [ ]:
cells = cells.astype(np.float32)
labels = labels.astype(np.int32)
cells = cells/255

In [ ]:
from sklearn.model_selection import train_test_split

train_x , x , train_y , y = train_test_split(cells , labels , 
                                            test_size = 0.2 ,
                                            random_state = 111)

eval_x , test_x , eval_y , test_y = train_test_split(x , y , 
                                                    test_size = 0.5 , 
                                                    random_state = 111)

In [ ]:
plt.figure(1 , figsize = (15 ,5))
n = 0 
for z , j in zip([train_y , eval_y , test_y] , ['train labels','eval labels','test labels']):
    n += 1
    plt.subplot(1 , 3  , n)
    sns.countplot(x = z )
    plt.title(j)
plt.show()

In [ ]:
print('train data shape {} ,eval data shape {} , test data shape {}'.format(train_x.shape,
                                                                           eval_x.shape ,
                                                                           test_x.shape))

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50, (7, 7), activation='relu', input_shape=(50, 50, 3), padding='same'),
    tf.keras.layers.Conv2D(90, (3, 3), activation='relu', padding='valid'),
    tf.keras.layers.Conv2D(10, (5, 5), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(5, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # Change to 2 for binary classification
])

# Compile the model with the correct learning rate parameter
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.001),  # Corrected here
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
checkpoint_path = "/tmp/modelchkpt/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_x, train_y,
          epochs=10,  
          validation_data=(eval_x, eval_y),
          callbacks=[cp_callback]) 

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

predicted_probabilities = model.predict(test_x)

predicted_labels = np.argmax(predicted_probabilities, axis=1)

cm = confusion_matrix(test_y, predicted_labels)

report = classification_report(test_y, predicted_labels)

accuracy = accuracy_score(test_y, predicted_labels)

print("Confusion Matrix:\n{}\n".format(cm))
print("Classification Report:\n{}\n".format(report))
print("Accuracy Score: {:.2f}%".format(accuracy * 100))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Plot setup
plt.figure(1, figsize=(15, 9))
n = 0 

# Loop to plot 49 random images from the test set
for i in range(49):
    n += 1
    # Randomly select an index from the test set
    r = np.random.randint(0, test_x.shape[0], 1)
    
    # Create a 7x7 grid of subplots
    plt.subplot(7, 7, n)
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    
    # Display the selected image
    plt.imshow(test_x[r[0]])
    
    # Set the title to show the true label and predicted label
    plt.title('True: {} | Pred: {}'.format(test_y[r[0]], predicted_labels[r[0]]))
    
    # Remove the x and y ticks for clarity
    plt.xticks([]), plt.yticks([])

# Show the plot
plt.show()
